In [252]:
!pip install ortools
!pip install pandas
import random
import pandas as pd
from ortools.linear_solver import pywraplp



In [253]:
import os

folder_path = r"C:\Users\mikie\OneDrive\שולחן העבודה\Fair_Schedule_App\OR-Tools fair scheduling\majd easy 4"
# Replace with the path to your folder
i=0
file_list = os.listdir(folder_path)
data_list = []

for file_name in file_list:
    file_path = os.path.join(folder_path, str(i)+'.txt')
    if file_name.endswith(".txt"):
        with open(file_path, "r") as file:
            contents = file.read()
            # Remove the quotes from the keys "0" and "1"
            contents = contents.replace('"0":', '0:')
            contents = contents.replace('"1":', '1:')
            contents = contents.replace('"2":', '2:')
            contents = contents.replace('"3":', '3:')
            contents = contents.replace('"4":', '4:')
            contents = contents.replace('"5":', '5:')
            contents = contents.replace('"6":', '6:')
            contents = contents.replace('"7":', '7:')
            contents = contents.replace('"8":', '8:')
            contents = contents.replace('"9":', '9:')
            contents = contents.replace('"10":', '10:')
            contents = contents.replace('"11":', '11:')



            i=i+1


            data_list.append(eval(contents))

print(data_list)


[{0: {0: [3, 2, 2, 1, 2, 2, 1], 1: [2, 2, 2, 1, 3, 2, 2], 2: [1, 1, 2, 3, 3, 1, 1]}, 1: {0: [2, 1, 2, 3, 2, 2, 3], 1: [1, 1, 2, 2, 1, 2, 1], 2: [2, 2, 2, 2, 2, 2, 2]}, 2: {0: [2, 3, 1, 1, 2, 2, 2], 1: [1, 1, 3, 1, 2, 2, 1], 2: [1, 2, 1, 2, 2, 2, 2]}, 3: {0: [1, 2, 1, 2, 3, 2, 1], 1: [1, 1, 2, 1, 2, 1, 2], 2: [2, 3, 1, 1, 3, 1, 2]}}]


In [254]:
from pandas.core.series import FillnaOptions


def algorithm(shift_requests):
    # Define variables
    num_workers = 4
    num_shifts = 3
    num_days = 7
    shifts = range(num_shifts)
    workers = range(num_workers)
    days = range(num_days)

    solver = pywraplp.Solver('shift_scheduling', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # Define binary variables indicating whether a worker is assigned to a shift or not
    assigned = {}
    for w in workers:
        for s in shifts:
            for d in days:
                assigned[w, s, d] = solver.BoolVar('worker %i assigned to shift %i on day %i' % (w, s, d))


    # Define objective function
    objective = solver.Sum([assigned[w, s, d] * shift_requests[w][s][d] for w in workers for s in shifts for d in days])



    # Add a new constraint that each worker is assigned to at most one shift per day
    for w in workers:
        for d in days:
            solver.Add(solver.Sum([assigned[w, s, d] for s in shifts]) <= 1)

    # Add a new constraint that each shift has exactly 1 worker assigned to it
    for s in shifts:
        for d in days:
            solver.Add(solver.Sum([assigned[w, s, d] for w in workers]) == 1)

    # Add a new constraint that a worker cannot work morning shift after a night shift on the previous day
    for w in workers:
        for d in range(1, num_days):  # Exclude the first day since there is no previous day
            solver.Add(assigned[w, 0, d] <= 1 - assigned[w, 2, d - 1])

    # Add a new constraint that a worker cannot work in a shift with priority 3
    for w in workers:
        for s in shifts:
            for d in days:
                if shift_requests[w][s][d] == 3:
                    solver.Add(assigned[w, s, d] == 0)

    solver.Minimize(objective)

    days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    shift_names = ["Morning", "Afternoon", "Night"]

    schedule_found = False
    while not schedule_found:
        status = solver.Solve()

        # Check if a feasible schedule is generated
        if status == pywraplp.Solver.OPTIMAL:
            with open(r"C:\Users\mikie\OneDrive\שולחן העבודה\Fair_Schedule_App\OR-Tools fair scheduling\result.txt", 'a') as file:
                file.write(str(solver.Objective().Value())+'\n')
            print("Objective value: ", solver.Objective().Value())
            print("Feasible schedule found.")
            schedule_found = True
        else:
            print("No feasible schedule found. Trying again...")
            with open(r"C:\Users\mikie\OneDrive\שולחן העבודה\Fair_Schedule_App\OR-Tools fair scheduling\result.txt", 'a') as file:
                file.write('0'+'\n')
            break

    # Create a dictionary to store the shift assignments for each worker
    worker_shifts = {}

    # Loop over each worker and add their shift assignments to the dictionary
    for w in workers:
        shift_assignments = []
        for d in days:
            for s in shifts:
                if assigned[w, s, days.index(d)].solution_value() == 1:
                    shift_assignments.append(shift_names[s])
                    break  # stop searching for this day's shift after the worker has been assigned
            else:
                shift_assignments.append('')  # worker was not assigned to any shift on this day

        worker_shifts['Worker ' + str(w)] = shift_assignments

    # worker_shifts visualization
    # Create an empty dataframe with the days of the week as columns and worker names as rows
    df_shifts = pd.DataFrame(index=["Worker {}".format(i) for i in range(num_workers)], columns=days)
    df_shifts = df_shifts.fillna('/')

    # Iterate over the dictionary to fill in the values
    for worker, shifts in worker_shifts.items():
        for i, shift in enumerate(shifts):
            if shift:
                df_shifts.loc[worker, days[i]] = shift

    # Apply formatting to the dataframe
    df_shifts_styled = df_shifts.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'center')]
    }])

    return df_shifts_styled





In [255]:
import pandas as pd
import numpy as np
def visualize_request(data):
    days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    shift_names = ["Morning", "Afternoon", "Night"]
    num_workers = 4
    # Calculate the number of shifts
    num_shifts = 3

    
    # Initialize the 2D array with zeros
    preferences = [[0] * (num_shifts *7) for _ in range(num_workers)]

    # Convert the data structure to a 2D array
    for worker_id, worker_data in data.items():
        for shift, priorities in worker_data.items():
            for day in range(len(priorities)):
                index = (day * 3) + int(shift)
                preferences[int(worker_id)][index] = priorities[day]

    # Print the resulting 2D array
    #print(preferences)
    requests_df = np.array(preferences.copy())
    requests_df = np.transpose(requests_df.reshape((num_shifts, 7, num_workers)), (0,1,2))

    # Flatten the column headers
    column_headers = [f'{day} {shift}' for day in days for shift in shift_names]

    # Create a DataFrame using pandas
    df = pd.DataFrame(requests_df.reshape((num_workers, num_shifts * 7)), columns=column_headers)
    # Modify the row labels
    df.index = [f'Worker {i}' for i in range(num_workers)]
    df.index.isin
    # Set pandas options to display all columns and rows without truncation
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.expand_frame_repr', False)

    # Display the DataFrame
    display(df)






In [256]:

for schedule in data_list:
  print(schedule)
  visualize_request(schedule)
  df_shifts = algorithm(schedule)
df_shifts


{0: {0: [3, 2, 2, 1, 2, 2, 1], 1: [2, 2, 2, 1, 3, 2, 2], 2: [1, 1, 2, 3, 3, 1, 1]}, 1: {0: [2, 1, 2, 3, 2, 2, 3], 1: [1, 1, 2, 2, 1, 2, 1], 2: [2, 2, 2, 2, 2, 2, 2]}, 2: {0: [2, 3, 1, 1, 2, 2, 2], 1: [1, 1, 3, 1, 2, 2, 1], 2: [1, 2, 1, 2, 2, 2, 2]}, 3: {0: [1, 2, 1, 2, 3, 2, 1], 1: [1, 1, 2, 1, 2, 1, 2], 2: [2, 3, 1, 1, 3, 1, 2]}}


,Sunday Morning,Sunday Afternoon,Sunday Night,Monday Morning,Monday Afternoon,Monday Night,Tuesday Morning,Tuesday Afternoon,Tuesday Night,Wednesday Morning,Wednesday Afternoon,Wednesday Night,Thursday Morning,Thursday Afternoon,Thursday Night,Friday Morning,Friday Afternoon,Friday Night,Saturday Morning,Saturday Afternoon,Saturday Night
Worker 0,3,2,1,2,2,1,2,2,2,1,1,3,2,3,3,2,2,1,1,2,1
Worker 1,2,1,2,1,1,2,2,2,2,3,2,2,2,1,2,2,2,2,3,1,2
Worker 2,2,1,1,3,1,2,1,3,1,1,1,2,2,2,2,2,2,2,2,1,2
Worker 3,1,1,2,2,1,3,1,2,1,2,1,1,3,2,3,2,1,1,1,2,2


Objective value:  25.0
Feasible schedule found.


,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
Worker 0,/,Night,/,Morning,Morning,Night,Night
Worker 1,Afternoon,Morning,Afternoon,/,Afternoon,Morning,Afternoon
Worker 2,Night,/,Night,Afternoon,Night,/,/
Worker 3,Morning,Afternoon,Morning,Night,/,Afternoon,Morning


In [257]:
def calculate_average(file_path):
    numbers = []
    count_2147483647 = 0

    # Read numbers from the file
    with open(file_path, 'r') as file:
        for line in file:
            number = float(line.strip())
            if number == 0:
                count_2147483647 += 1
            else:
                numbers.append(number)

    # Calculate the average
    average = sum(numbers) / len(numbers)

    # Print the average and count of occurrences of 2147483647
    print(f"Average: {average}")
    print(f"Count of 2147483647: {count_2147483647}")


# Specify the path to your input file
file_path = r"C:\Users\mikie\OneDrive\שולחן העבודה\Fair_Schedule_App\OR-Tools fair scheduling\result.txt"

# Call the function with the file path
calculate_average(file_path)

Average: 26.45945945945946
Count of 2147483647: 2
